<a href="https://colab.research.google.com/github/kjcoursera/TensorFlow/blob/main/Classify_images_of_clothing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>